In [2]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=3afb51afad87a473d9e9d39dcfaf3ac5ce51b2fc461cdba78f9af51fd09c7e52
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from bayes_opt import BayesianOptimization

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Hyper-parameters
num_epochs = 10
batch_size = 1

In [6]:
class IrisDataset(Dataset):

    # data loading
    def __init__(self):
        iris = datasets.load_iris()
        feature = pd.DataFrame(iris.data, columns=iris.feature_names)
        target = pd.DataFrame(iris.target, columns=['target'])
        iris_data = pd.concat([target, feature], axis=1)
        # Data type change and flatten targets
        self.x = torch.from_numpy(np.array(iris_data)[:, 1:].astype(np.float32))
        self.y = torch.from_numpy(np.array(iris_data)[:, [0]].astype(np.longlong).flatten())
        self.n_samples = self.x.shape[0]

    # working for indexing
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]

    # return the length of our dataset
    def __len__(self):

        return self.n_samples


dataset = IrisDataset()

# create data spliter
def dataSplit(dataset, val_split=0.2, shuffle=False, random_seed=0):

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    train_indices, val_indices = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    return train_sampler , valid_sampler


train_sampler, valid_sampler = dataSplit(dataset = dataset, val_split = 0.2, shuffle = True, random_seed = 0)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [7]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=4, out_features=24)
        self.fc2 = nn.Linear(in_features=24, out_features=24)
        self.fc3 = nn.Linear(in_features=24, out_features=3)

        
    def forward(self, net):
        net = F.relu(self.fc1(net))
        net = F.relu(self.fc2(net))
        net = self.fc3(net)

        return net

model = Network()
model.to(device)

Network(
  (fc1): Linear(in_features=4, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=3, bias=True)
)

In [8]:
def train(lrate):
  model = Network()
  model.to(device)
  lossfunc = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=lrate)

  for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)
      
      outputs = model(images)
      loss = lossfunc(outputs, labels)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
    print('In epoch {}, loss: {:.3f}'.format(
             epoch+1, train_loss/len(train_loader)))

            
  model.eval()
  n_correct = 0
  n_sample = 0
  with torch.no_grad():
    for (images, labels) in (val_loader):
      images = images.to(device)
      labels = labels.to(device)

      predicted = model(images)
      predicted = torch.argmax(predicted,1)
      n_correct += ( labels.view(-1) == predicted ).sum().item()
  
    return n_correct/len(val_loader)

In [9]:
pbounds = {'lrate': (1e-5, 1e-1)}

optimizer = BayesianOptimization(
    f=train,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)
optimizer.maximize(n_iter=10)


print(optimizer.max)

|   iter    |  target   |   lrate   |
-------------------------------------
In epoch 1, loss: 0.864
In epoch 2, loss: 0.428
In epoch 3, loss: 0.419
In epoch 4, loss: 0.435
In epoch 5, loss: 0.418
In epoch 6, loss: 0.335
In epoch 7, loss: 0.372
In epoch 8, loss: 0.385
In epoch 9, loss: 0.339
In epoch 10, loss: 0.345
|  1        |  0.9667   |  0.04171  |
In epoch 1, loss: 0.721
In epoch 2, loss: 0.507
In epoch 3, loss: 0.522
In epoch 4, loss: 0.438
In epoch 5, loss: 0.409
In epoch 6, loss: 0.401
In epoch 7, loss: 0.420
In epoch 8, loss: 0.521
In epoch 9, loss: 0.460
In epoch 10, loss: 0.362
|  2        |  0.8      |  0.07204  |
In epoch 1, loss: 1.240
In epoch 2, loss: 1.224
In epoch 3, loss: 1.210
In epoch 4, loss: 1.197
In epoch 5, loss: 1.185
In epoch 6, loss: 1.175
In epoch 7, loss: 1.164
In epoch 8, loss: 1.155
In epoch 9, loss: 1.146
In epoch 10, loss: 1.139
|  3        |  0.2333   |  2.144e-0 |
In epoch 1, loss: 0.853
In epoch 2, loss: 0.451
In epoch 3, loss: 0.379
In epoch 4, los

In [11]:
params = optimizer.max["params"]
print(params['lrate'])
model_acc = train(params['lrate'])
print("model_test_acc:" , model_acc)

0.014684121522803134
In epoch 1, loss: 0.986
In epoch 2, loss: 0.657
In epoch 3, loss: 0.484
In epoch 4, loss: 0.430
In epoch 5, loss: 0.394
In epoch 6, loss: 0.385
In epoch 7, loss: 0.338
In epoch 8, loss: 0.310
In epoch 9, loss: 0.319
In epoch 10, loss: 0.278
model_test_acc: 1.0
